<a href="https://colab.research.google.com/github/MrRoboto11102003/RL-Project/blob/main/RLConnect4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle has a basic struture of the connect 4 board. (installing kaggle might take several tries).


In [ ]:
!pip install kaggle_environments


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached kaggle_environments-1.13.0-py2.py3-none-any.whl (1.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.5 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!mkdir -p saved_model

In [ ]:
import tensorflow as tf
from keras import datasets, layers, models
from keras.layers import Dense,Flatten
import numpy as np
from kaggle_environments import evaluate, make, utils

#kaggle_connect4 board used for training the agent 

def check_if_done(observation):
    done = [False,'No Winner Yet']
    #horizontal check
    for i in range(6):
        for j in range(4):
            if observation[i][j] == observation[i][j+1] == observation[i][j+2] == observation[i][j+3] == 1:
                done = [True,'Player 1 Wins Horizontal']
            if observation[i][j] == observation[i][j+1] == observation[i][j+2] == observation[i][j+3] == 2:
                done = [True,'Player 2 Wins Horizontal']
    #vertical check
    for j in range(7):
        for i in range(3):
            if observation[i][j] == observation[i+1][j] == observation[i+2][j] == observation[i+3][j] == 1:
                done = [True,'Player 1 Wins Vertical']
            if observation[i][j] == observation[i+1][j] == observation[i+2][j] == observation[i+3][j] == 2:
                done = [True,'Player 2 Wins Vertical']
    #diagonal check top left to bottom right
    for row in range(3):
        for col in range(4):
            if observation[row][col] == observation[row + 1][col + 1] == observation[row + 2][col + 2] == observation[row + 3][col + 3] == 1:
                done = [True,'Player 1 Wins Diagonal']
            if observation[row][col] == observation[row + 1][col + 1] == observation[row + 2][col + 2] == observation[row + 3][col + 3] == 2:
                done = [True,'Player 2 Wins Diagonal']
    
    #diagonal check bottom left to top right
    for row in range(5, 2, -1):
        for col in range(3):
            if observation[row][col] == observation[row - 1][col + 1] == observation[row - 2][col + 2] == observation[row - 3][col + 3] == 1:
                done = [True,'Player 1 Wins Diagonal']
            if observation[row][col] == observation[row - 1][col + 1] == observation[row - 2][col + 2] == observation[row - 3][col + 3] == 2:
                done = [True,'Player 2 Wins Diagonal']
    return done

def create_model():
    model = models.Sequential()

    # Input is the board.
    # Consider 70 n-tuples of length 8. This gives us 560 intial weights, followed by 70 n tuple weights, This then gives us our ourput
    
    model.add(Flatten())
    model.add(Dense(560, activation='relu',input_shape=(1,6,7,1)))
    model.add(Dense(70, activation='relu'))
    
    model.add(Dense(7))
    
    return model
   
def compute_loss(logits, actions, rewards): 
    neg_logprob = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=actions)
    loss = tf.reduce_mean(neg_logprob * rewards)
    return loss
  
def train_step(model, optimizer, observations, actions, rewards):
    with tf.GradientTape() as tape:
      # Forward propagate through the agent network
        
        logits = model(observations)
        loss = compute_loss(logits, actions, rewards)
        grads = tape.gradient(loss, model.trainable_variables)
        
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

def get_action(model, observation, epsilon):
    #determine whether model action or random action based on epsilon
    act = np.random.choice(['model','random'], 1, p=[1-epsilon, epsilon])[0]
    observation = np.array(observation).reshape(1,6,7,1)
    logits = model.predict(observation)
    prob_weights = tf.nn.softmax(logits).numpy()
    
    if act == 'model':
        action = list(prob_weights[0]).index(max(prob_weights[0]))
    if act == 'random':

        #Improvement implemented here
        #action = np.random.choice(7) old distribution
        action = np.random.binomial(6,0.5) #new distribution
        
    return action, prob_weights[0]

def check_if_action_valid(obs,action):
    if obs[action] == 0:
        
        valid = True
    else:
        valid = False
    return valid

def player_1_agent(observation, configuration):
    action, prob_weights = get_action(player_1_model,observation['board'],0)
    if check_if_action_valid(observation['board'],action):
        return action
    else:
        while True:
            previous_prob_weight = prob_weights[action]
            temp_prob = min(prob_weights)
            for prob in prob_weights:
                if prob < previous_prob_weight and prob > temp_prob:
                    temp_prob = prob
                    action = list(prob_weights).index(temp_prob)
            if check_if_action_valid(observation['board'],action):
                break
            
    return action

class Memory:
    def __init__(self): 
        self.clear()

    # Resets/restarts the memory buffer
    def clear(self): 
        self.observations = []
        self.actions = []
        self.rewards = []
        self.info = []
        
    def add_to_memory(self, new_observation, new_action, new_reward): 
        self.observations.append(new_observation)
        self.actions.append(new_action)
        self.rewards.append(float(new_reward))

#train player 1 against random agent
tf.keras.backend.set_floatx('float64')
optimizer = tf.keras.optimizers.Adam(0.01)
optimizer2 = tf.keras.optimizers.Adam(0.01)

win_count=0

env = make("connectx", debug=True)
memory = Memory()
epsilon = 1

#player_1_model=tf.keras.models.load_model('saved_model/player_1_model')

#player_2_model=tf.keras.models.load_model('saved_model/player_2_model')


#for new model use bellow
player_1_model=create_model()
player_1_model.build((1,6,7,1))

player_2_model=create_model()
player_2_model.build((1,6,7,1))

wins=[0,0,0,0,0,0,0,0,0,0]

for i_episode in range(6000):

    if i_episode==100:
      wins[0]=win_count
    if i_episode==500:
      wins[1]=win_count
    if i_episode==1000:
      wins[2]=win_count
    if i_episode==2000:
      wins[3]=win_count
    if i_episode==3000:
      wins[4]=win_count
    if i_episode==4000:
      wins[5]=win_count
    if i_episode==5999:
      wins[6]=win_count
    
    trainer = env.train([None,'random'])
        
    observation = trainer.reset()['board']
    memory.clear()
    epsilon = epsilon * .998
    overflow = False
    while True:
        action, _ = get_action(player_1_model,observation,epsilon)
        next_observation, dummy, overflow, info = trainer.step(action)
        observation = next_observation['board']
        observation = [float(i) for i in observation]
        done = check_if_done(np.array(observation).reshape(6,7))
        
        #Rewards
        if done[0] == False:
            reward = 0
        if 'Player 2' in done[1]:
            reward = -1
        if 'Player 1' in done[1]:
            win_count += 1
            reward = 1
        if overflow == True and done[0] == False:
            reward = -99
            done[0] = True
        
        memory.add_to_memory(np.array(observation).reshape(6,7,1), action, reward)
        if done[0]:
            #train after each game
            
            train_step(player_2_model, optimizer,
                     observations=np.array(memory.observations),
                     actions=np.array(memory.actions),
                     rewards = memory.rewards)
            train_step(player_1_model, optimizer2,
                     observations=np.array(memory.observations),
                     actions=np.array(memory.actions),
                     rewards = memory.rewards)
            
            break

player_1_model.summary()
player_1_model.save('saved_model/player_1_model')
player_2_model.save('saved_model/player_2_model')
print(win_count*100/(i_episode+1))

print(wins[0]*100/100," 100 eps")
print(wins[1]*100/500," 500 eps")
print(wins[2]*100/1000," 1000 eps")
print(wins[3]*100/2000," 2000 eps")
print(wins[4]*100/3000," 3000 eps")
print(wins[5]*100/4000," 4000 eps")
print(wins[6]*100/6000," 6000 eps")



Loading environment lux_ai_s2 failed: No module named 'vec_noise'
1/1 [==============================] - 0s 16ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 19ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 30ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 22ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 23ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 18ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 26ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 28ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 27ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 43ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 27ms/step
Invalid Action: Invalid column: 1
1/1 [==============================] - 0s 23ms/step
Invalid Action: Invalid column: 1
1/1

68.78787878787878
67.0  100 eps
57.0  500 eps
61.09090909090909  1000 eps
64.81818181818181  2000 eps
67.57575757575758  3000 eps
68.20454545454545  4000 eps
68.77272727272727  6000 eps
